## Lab 02 - Parte 1 - Expansão de Consultas

Nesta atividade você vai exercitar a noção de expansão de consultas. Considerando a coleção de notícias do lab passado, você primeiro precisa executar os seguintes passos:

1. Escreva uma função que receba uma coleção de documentos e retorne uma matrix de termos-termos contendo as frequências de co-ocorrência de duas palavras consecutivas no texto (bigramas).
2. Escreva uma função que receba um certo termo de consulta e a matriz construída no passo 1 acima e retorneas top-3 palavras em ordem decrescente de frequencia.
3. Expanda a consulta original com os termos retornados no passo 2 acima.
4. Faça uma busca disjuntiva (OR) considerando a nova consulta.

Por exemplo, se o termo de consulta é petrobrás e as palavras que mais co-ocorrem com petrobrás são: empresa, gasolina e estatal, a consulta agora ficaria: petrobrás OR empresa OR gasolina OR estatal.

Escolha livremente três termos de consulta e responda o seguinte:

- Quais os termos retornados para a expansão de cada consulta?
- Você acha que esses termos são de fato relacionados com a consulta original? Justifique.
- Compare os documentos retornados para a consulta original com a consulta expandida. Quais resultados você acha que melhor capturam a necessidade de informação do usuário? Por que?
- A expansão de consultas é mais adequada para melhorar o recall ou o precision? Por que?

### Solução

#### Importando bibliotecas

In [69]:
import pandas as pd
import numpy as np
import collections
from nltk.tokenize import word_tokenize

#### Constantes

In [70]:
FILE_PATH = 'data/estadao_noticias_eleicao.csv'
ROWS_NUMBER = 10
NOTICIA_COL = "noticia"
CONTEUDO_COL = "conteudo"
TITULO_COL = "titulo"
SUBTITULO_COL = "subTitulo"
VALOR_AXIS = 1
ID_NOTICIA_COL = "idNoticia"
TOKENS_COL = "tokens"

#### Funções

In [71]:
def concatena_dados(row):
    return (row[TITULO_COL] + " " + row[SUBTITULO_COL] + " " + row[CONTEUDO_COL]).lower()

def tokeniza(conteudo):
    stop_words = set(stopwords.words('portuguese'))
    return set([w for w in word_tokenize(conteudo) if not w in stop_words])

#### Importando dados

In [72]:
df = pd.read_csv(FILE_PATH, encoding = 'utf-8')
df = df.replace(np.NAN, "")

In [73]:
df.head(n=ROWS_NUMBER)

,timestamp,titulo,subTitulo,conteudo,url,idNoticia
0,2014-12-31T00:00:00Z,PT espera 30 mil pessoas em festa na Esplanada,Objetivo é demonstrar apoio popular a Dilma e ...,BRASÍLIA - Após o desgaste provocado com o lan...,"http://politica.estadao.com.br/noticias/geral,...",1
1,2014-12-31T00:00:00Z,Alckmin toma posse de olho no Planalto,Governador reeleito tenta amarrar tucanos paul...,"Reeleito em outubro, o governador tucano Geral...","http://politica.estadao.com.br/noticias/geral,...",2
2,2014-12-31T00:00:00Z,Seis obstáculos e desafios do segundo mandato ...,"Em meio a escândalo de corrupção, presidente t...",1. Rearranjo das contas A nova equipe econôm...,"http://politica.estadao.com.br/noticias/geral,...",3
3,2014-12-31T00:00:00Z,,Veja as principais fotos do dia e dos eventos ...,,"http://fotos.estadao.com.br/fotos/politica,dil...",4
4,2014-12-31T00:00:00Z,,Veja as principais fotos do dia e dos eventos ...,,"http://fotos.estadao.com.br/fotos/politica,dil...",5
5,2014-12-31T00:00:00Z,,Veja as principais fotos do dia e dos eventos ...,,"http://fotos.estadao.com.br/fotos/politica,dil...",6
6,2014-12-31T00:00:00Z,Veja os desafios dos governadores que assumem ...,,"No Acre, governador reeleito quer erradicar an...","http://politica.estadao.com.br/noticias/geral,...",7
7,2014-12-31T00:00:00Z,PT impulsiona cerimônia de posse da Dilma nas ...,"Já entre os 27 governadores eleitos, apenas se...","Os perfis da presidente Dilma Rousseff, nas re...","http://politica.estadao.com.br/noticias/geral,...",8
8,2014-12-31T00:00:00Z,,"Em setembro de 2014, após conceder entrevista ...",,"http://tv.estadao.com.br/videos,politica,dilma...",9
9,2014-12-31T00:00:00Z,Dilma veta projeto de Suplicy que cria linha o...,Proposta havia sido aprovada no Senado depois ...,A presidente Dilma Rousseff vetou integralment...,"http://politica.estadao.com.br/noticias/geral,...",10


#### Tratando dados

In [74]:
df[NOTICIA_COL] = df.apply(lambda row: concatena_dados(row), axis=VALOR_AXIS)

In [75]:
df = df[[ID_NOTICIA_COL, NOTICIA_COL]]

In [76]:
df.head(n=ROWS_NUMBER)

,idNoticia,noticia
0,1,pt espera 30 mil pessoas em festa na esplanada...
1,2,alckmin toma posse de olho no planalto governa...
2,3,seis obstáculos e desafios do segundo mandato ...
3,4,veja as principais fotos do dia e dos eventos...
4,5,veja as principais fotos do dia e dos eventos...
5,6,veja as principais fotos do dia e dos eventos...
6,7,veja os desafios dos governadores que assumem ...
7,8,pt impulsiona cerimônia de posse da dilma nas ...
8,9,"em setembro de 2014, após conceder entrevista..."
9,10,dilma veta projeto de suplicy que cria linha o...


In [77]:
df[TOKENS_COL] = df.apply(lambda row: tokeniza(row[NOTICIA_COL]), axis=VALOR_AXIS)

In [78]:
df.head(n=ROWS_NUMBER)

,idNoticia,noticia,tokens
0,1,pt espera 30 mil pessoas em festa na esplanada...,"{noite, após, governo, duvido, 10, caravana, m..."
1,2,alckmin toma posse de olho no planalto governa...,"{plano, governo, articulação, 44, secretariado..."
2,3,seis obstáculos e desafios do segundo mandato ...,"{qualidade, econômica, governo, carros, saúde,..."
3,4,veja as principais fotos do dia e dos eventos...,"{dia, ., estadão, brasil, veja, principais, mu..."
4,5,veja as principais fotos do dia e dos eventos...,"{dia, ., estadão, brasil, veja, principais, mu..."
5,6,veja as principais fotos do dia e dos eventos...,"{dia, ., estadão, brasil, veja, principais, mu..."
6,7,veja os desafios dos governadores que assumem ...,"{econômica, porém, dezembro, lrf, popular, pro..."
7,8,pt impulsiona cerimônia de posse da dilma nas ...,"{assessoria, tentaram, apesar, dezembro, gomes..."
8,9,"em setembro de 2014, após conceder entrevista...","{apresentou, após, tv, sobre, setembro, jornal..."
9,10,dilma veta projeto de suplicy que cria linha o...,"{tramitou, após, governo, veto, apesar, plano,..."


#### 1. Matriz de Ocorrência

In [79]:
'''Este código foi copiado de https://github.com/allansales/information-retrieval/blob/master/Lab%202/coocurrence_matrix.ipynb'''

from scipy import sparse
import nltk
from nltk import bigrams
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer

def co_occurrence_matrix(corpus):
    vocab = set(corpus)
    vocab = list(vocab)
    n = len(vocab)
   
    vocab_to_index = {word:i for i, word in enumerate(vocab)}
    
    bi_grams = list(bigrams(corpus))

    bigram_freq = nltk.FreqDist(bi_grams).most_common(len(bi_grams))

    I=list()
    J=list()
    V=list()
    
    for bigram in bigram_freq:
        current = bigram[0][1]
        previous = bigram[0][0]
        count = bigram[1]

        I.append(vocab_to_index[previous])
        J.append(vocab_to_index[current])
        V.append(count)
        
    co_occurrence_matrix = sparse.coo_matrix((V,(I,J)), shape=(n,n))

    return co_occurrence_matrix, vocab_to_index

tokenizer = RegexpTokenizer(r'\w+')
tokens_lists = df.noticia.apply(lambda text: tokenizer.tokenize(text.lower()))

stopword_ = stopwords.words('portuguese')
filtered_tokens = tokens_lists.apply(lambda tokens: [token for token in tokens if token not in stopword_])

tokens = [token for tokens_list in filtered_tokens for token in tokens_list]
matrix, vocab = co_occurrence_matrix(tokens)

consultable_matrix = matrix.tocsr()

def consult_frequency(w1, w2):
    return(consultable_matrix[vocab[w1],vocab[w2]])

w1 = 'poucos'
w2 = 'recursos'
consult_frequency(w1, w2)

3

#### 2. Função que retorna top 3 palavras mais frequentes baseado em #1

In [80]:
# invert key value 
util_vocab = {vocab[key]: key for key in vocab}

def top_words(termo, N=3):
    n_array = np.reshape(consultable_matrix[termo].toarray(), -1)
    n_dict = {}
    for i in (-n_array).argsort()[:N]:
        n_dict[util_vocab[i]] = n_array[i]
    return list(n_dict.keys())

#### 3. Função que expande consulta

In [81]:
def expande_query(termos):
    query = []
    termos = termos.split(' ')
    if len(termos) > 1:
        for termo in termos:
            query.append(termo)
            query.extend(top_words(vocab[termo]))
    else:
        query.append(termos[0])
        query.extend(top_words(vocab[termos[0]]))
        
    return " ".join(str(x) for x in set(query))

#### 4. Consulta OR



In [104]:
def criar_indexes(tokens, docId):
    for token in tokens:
        if token in d:
            if docId in d[token]: d[token][docId] += 1
            else: d[token][docId] = 1
        else:
            d[token] = {}
            d[token][docId] = 1

def busca(consulta):
    consulta = consulta.lower().split(' ')
    if len(consulta) >= 1:
        termos = [termo for termo in consulta if termo != 'OR']
        r = set(d[termos[0]])
        for t in termos[1:]:
            r = r | set(d[t])
        return list(r)
    return []

d = collections.defaultdict(list)

for index, row in df.iterrows():  
    criar_indexes(row.tokens, row.idNoticia)

## Respostas

Escolhendo três termos livremente, temos:
- Corrupção
- Eleição
- Lula

In [124]:
q = ["corrupção", "eleição", "lula"]
exp_q = []

for t in q:
    print(t)
    r = expande_query(t)
    exp_q.append(r)
    print(r)

corrupção
corrupção passiva petrobrás é
eleição
2010 presidencial eleição é
lula
silva disse dilma lula


De acordo com o resultado acima, é  possível ver que a query original está relacionada com a query expandida.
Corrupção está fortemente associado à Petrobrás dados os últimos escandalos, assim como Dilma está relacionada com eleicao presidencial, alem disto, Lula e Dilma estao associados tambem.

In [136]:
for i in range(len(q)):
    print("Consulta para %s retornou %d documentos" % (q[i], len(busca(q[i]))))
    print("Consulta expandida para %s (%s) retornou %d documentos" % (q[i], exp_q[i], len(busca(exp_q[i]))))
    print()

Consulta para corrupção retornou 1138 documentos
Consulta expandida para corrupção (corrupção passiva petrobrás é) retornou 6914 documentos

Consulta para eleição retornou 1792 documentos
Consulta expandida para eleição (2010 presidencial eleição é) retornou 6932 documentos

Consulta para lula retornou 1891 documentos
Consulta expandida para lula (silva disse dilma lula) retornou 6402 documentos



Os resultados que mais atenderao as necessidades do usuario serao os da consulta expandida pois visto que a consulta expandida esta relacionada a consulta original e que a consulta expandida retornam muito mais documentos, entao certamente havera informacao mais valiosa para o usuario.

Ainda sobre a consulta expandida, a mesma melhorara o recall e precision pois de acordo com https://en.wikipedia.org/wiki/Precision_and_recall precision é a fracao relevante de instancias dentre as instancias recuperadas e recall é a fracao de instancias relevantes que foram recuperadas sobre o total de instancias relevantes. Logo, se a quantidade de instancias recuperadas com a consulta expandida foi muito maior que a da consulta original, entao precision e recall aumentarao.